In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import joblib
from sklearn.neighbors import NearestNeighbors
from google.cloud import bigquery
pd.set_option('display.max_columns', None)


/Users/jordanwright/data_science_projects/spotify_recommendation_engine/.venv/lib/python3.9/site-packages/google/api_core/_python_version_support.py:246: FutureWarning: You are using a non-supported Python version (3.9.4). Google will not post any further updates to google.api_core supporting this Python version. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
/Users/jordanwright/data_science_projects/spotify_recommendation_engine/.venv/lib/python3.9/site-packages/google/auth/__init__.py:54: FutureWarning: You are using a Python version 3.9 past its end of life. Google will update google-auth with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade your Python version, and then update google-auth.
  warnings.warn(eol_message.format("3.9"), FutureWarning)
/Users/jordanwright/data_science_projects/spotify_recommendation_engine/.venv/lib/python3.9/

In [18]:
df = pd.read_csv('spotify_songs.csv')
df.head(15)

,track_id,track_name,artists,track_genre,danceability,energy,speechiness,acousticness,instrumentalness,valence,norm_popularity,norm_tempo,norm_loudness
0,3nqQXoyQOWXiESFLlDF1hG,Unholy (feat. Kim Petras),Sam Smith;Kim Petras,dance,0.714,0.472,0.0864,0.01300,0.000005,0.238,1.00,0.524484,0.877083
1,2tTmW7RDtMQtBk7m2rYeSw,"Quevedo: Bzrp Music Sessions, Vol. 52",Bizarrap;Quevedo,hip-hop,0.621,0.782,0.0440,0.01250,0.033000,0.550,0.99,0.512132,0.907533
2,5ww2BF9slyYgNOk37BlC4u,La Bachata,Manuel Turizo,latin,0.835,0.679,0.0364,0.58300,0.000002,0.850,0.98,0.499920,0.911183
3,4uUG5RXrOk84mYEfFvj3cK,I'm Good (Blue),David Guetta;Bebe Rexha,dance,0.561,0.965,0.0343,0.00383,0.000007,0.304,0.98,0.512160,0.938783
4,1IHWl5LamUGEuP4ozKQSXZ,Tití Me Preguntó,Bad Bunny,latin,0.650,0.715,0.2530,0.09930,0.000291,0.187,0.97,0.426688,0.913367
5,6Sq7ltF9Qa7SNFBsV5Cogx,Me Porto Bonito,Bad Bunny;Chencho Corleone,latin,0.911,0.712,0.0817,0.09010,0.000027,0.425,0.97,0.368020,0.914917
6,5Eax0qFko2dh7Rl2lYs3bx,Efecto,Bad Bunny,latin,0.801,0.475,0.0516,0.14100,0.000017,0.234,0.96,0.392188,0.853383
7,5IgjP7X4th6nMNDh4akUHb,Under The Influence,Chris Brown,dance,0.733,0.690,0.0427,0.06350,0.000001,0.310,0.96,0.467968,0.907850
8,4h9wh7iOZ0GGn8QVp4RAOB,I Ain't Worried,OneRepublic,piano,0.704,0.797,0.0475,0.08260,0.000745,0.825,0.96,0.559976,0.901217
9,4LRPiXqCikLlN15c3yImP7,As It Was,Harry Styles,pop,0.520,0.731,0.0557,0.34200,0.001010,0.662,0.95,0.695720,0.911033


In [8]:
feature_cols = [
    'danceability', 'energy', 'speechiness', 'acousticness', 
    'instrumentalness', 'valence', 'norm_popularity', 
    'norm_tempo', 'norm_loudness'
]

X = df[feature_cols]

In [20]:
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(X)
joblib.dump(model, 'music_model.joblib')
df.to_pickle('music_data.pkl')

In [21]:
df['energy_acoustic_ratio'] = df['energy'] / (df['acousticness'] + 0.1)


df['positivity_index'] = df['danceability'] * df['valence']

artist_counts = df['artists'].value_counts().to_dict()
df['artist_presence'] = df['artists'].map(artist_counts)

df.to_pickle('music_data.pkl')